In [1]:
from preproc_utils import *
from Get_PSSM import *
from Get_dataset import *
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
chebi = pd.read_table('/Users/suhancho/data/Uniprot_metalbinding_challenge/POS_TRAIN_FULL.tsv')
inpath = '/Users/suhancho/data/Uniprot_metalbinding_challenge/chebi/'
bind_tsv_list = [inpath + f for f in os.listdir(inpath)]
pssm_path = '/Users/suhancho/data/Uniprot_metalbinding_challenge/PSSM/'
pssm_files = [pssm_path+f for f in os.listdir(pssm_path)]

In [2]:
chebi = pd.read_table('/Users/suhancho/data/Uniprot_metalbinding_challenge/POS_TRAIN_FULL.tsv')
inpath = '/Users/suhancho/data/Uniprot_metalbinding_challenge/chebi/'
bind_tsv_list = [inpath + f for f in os.listdir(inpath)]
pssm_path = '/Users/suhancho/data/Uniprot_metalbinding_challenge/PSSM/'
pssm_files = [pssm_path+f for f in os.listdir(pssm_path)]

In [3]:
def calculate_window(num_inspections,bs_idx):
    train_dat=[]
    for i,pssm in tqdm(enumerate(pssm_files[0:num_inspections])):
        ion_file =bind_tsv_list[bs_idx] # bs_idx : 0~29
        ion_name = ion_file.split('/')[-1].split('.')[0]
        bs = get_binding_site(ion_file,pssm.split('/')[-1].split('.')[0])
        try : 
            if len(bs)!=0:
                gt,fs = get_dataset(get_processed_pssm(pssm),bs)
                train_dat.append([gt[0].values.tolist(),1])
                train_dat.append([fs[0].values.tolist(),0])
        except:
            print(pssm)

    return(train_dat,ion_name)

In [4]:
def preproc_data(windowdata):
    train_X = [dat[0] for dat in windowdata]
    train_Y = [dat[1] for dat in windowdata]
    print('Size of dataset : '+str(len(train_X)))
    return(train_X,train_Y)

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
def check_windowdata(traindata):
    sns.histplot([len(t) for t in traindata])
    plt.show()



In [6]:
def filter_traindata(Xdata,Ydata):
    train_X_filtered = [Xdata[i] for i in range(len(Xdata)) if len(Xdata[i])==9]
    train_Y_filtered = [Ydata[i] for i in range(len(Xdata)) if len(Xdata[i])==9]
    return(train_X_filtered,train_Y_filtered)  

In [7]:
from itertools import chain
def flatten_Xdata(filtered_X):
    flatten_trainX = [list(chain.from_iterable(lst)) for lst in filtered_X]
    return(flatten_trainX)

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score,f1_score
def get_MLmetrics(testset_y,testset_X,classifier,ion_name):
    auc = roc_auc_score(testset_y,list(classifier.predict(testset_X)))
    acc = accuracy_score(testset_y,list(classifier.predict(testset_X)))
    recall = recall_score(testset_y,list(classifier.predict(testset_X)))
    f1 = f1_score(testset_y,list(classifier.predict(testset_X)))
    prec = precision_score(testset_y,list(classifier.predict(testset_X)))
    return(print(ion_name,auc,acc,recall,f1,prec))

In [11]:
train_dat,ion = calculate_window(1000,11)
train_X,train_Y = preproc_data(train_dat)
check_windowdata(train_X)
train_X_filtered,train_Y_filtered = filter_traindata(train_X,train_Y)
flatten_trainX = flatten_Xdata(train_X_filtered)
trainX, testX, trainy, testy = train_test_split(flatten_trainX,train_Y_filtered,test_size=0.2,shuffle = True)

# flatten_trainX = flatten_Xdata(train_X)
# trainX, testX, trainy, testy = train_test_split(flatten_trainX,train_Y_filtered),test_size=0.2,shuffle = True)

from sklearn import svm
X = trainX
y = trainy

clf = svm.SVC()
clf.fit(X, y)
get_MLmetrics(testy,testX,clf,ion)

Zn(2+) 0.889983579638752 0.8925619834710744 0.8275862068965517 0.8807339449541284 0.9411764705882353
